# Label Dataset Program

In [2]:
import os
import cv2
import shutil
import numpy as np
import sys
from IPython.display import clear_output
from pynput.keyboard import Key

# Organise video into classes

## Sort Files

In [5]:
videoLocation = "D:/Thesis/Video-Segments-AUSLAN"
folderList = os.listdir(videoLocation)
words = {}
wordAmount = {}
videoNum = 0
for videoName in folderList:
    location = videoLocation + '/' + videoName
    videoList = os.listdir(location)
    for videoName in videoList:
        if (videoName.count('_')) == 3:
            videoNum += 1
            try:
                wordAmount[str.split(videoName,'_')[0]] += 1
                words[str.split(videoName,'_')[0]].append(location + '/' + videoName)
            except KeyError:
                wordAmount[str.split(videoName,'_')[0]] = 1
                words[str.split(videoName,'_')[0]] = [location + '/' + videoName]

KeyboardInterrupt: 

## Move files

In [ ]:
f = open("failedToSort.txt", "w")
saveLocation = "D:/Thesis/Video-Auslan-sorted"
for key in words.keys():
    if key == '':
        continue
    wordFolder = saveLocation + '/' + key
    try:
        os.mkdir(wordFolder)
    except OSError as error:
        print(key, 'folder exists')
    for i, videoPath in enumerate(words[key]):
        if os.path.isfile(wordFolder + '/' + words[key][i].split('/')[-1]):
            print(words[key][i].split('/')[-1], 'exists')
        else:
            if os.path.isfile(videoPath):
                try:
                    shutil.copy(videoPath, wordFolder)
                except:
                    f.write(videoPath + '\n')
            else:
                print('cannot copy')
    print(key, 'coppied')
f.close()

#### Check if moved correct

In [6]:
saveLocation = "D:/Thesis/Video-Auslan-sorted"
numVid = 0
for folders in os.listdir(saveLocation):
    numVid += len(os.listdir(saveLocation + '/' +folders))

numOriginal = 0
fromLocation = "D:/Thesis/Video-Segments-AUSLAN"
for folders in os.listdir(fromLocation):
    numOriginal += len(os.listdir(fromLocation + '/' + folders))
print('Number of videos Original: ', numOriginal)
print('Number of videos New: ', numVid)
#Loss of videos likley from old file locations had random dummy files 

KeyboardInterrupt: 

### ERRORED FILE

In [9]:
corrupted = ['AJBB1c2b',
'AJPB1c2b',
'AMMA1c2a',
'AWGB1c2a',
'BAOBB1c2a',
'BCPA1c2a',
'BDLA1c2a',
'BFPB1c2a',
'BFSA1c2a',
'BGLB1c2b',
'BRCA1c2a',
'BRVA1c2b',
'BSSA1c2b',
'BCHA2c7a',
'SSNA1c2a']

### Labeler For Path Movements

In [90]:
saveLocation = "D:/Thesis/Video-Auslan-sorted"
pathMovement = {}
for word in os.listdir(saveLocation):
    pathMovement[word] = None

In [91]:
saveLocation = "D:/Thesis/Video-Auslan-sorted"
# on a scale of 1 to 4
finishedWords = {}
for word in os.listdir(saveLocation):
    wordLocation = saveLocation + '/' + word
    # Loop through every video
    for videoName in os.listdir(wordLocation): 
        finishedWords[videoName] = False

In [93]:
saveLocation = "D:/Thesis/Video-Auslan-sorted"
# on a scale of 1 to 4
usability = {}
for word in os.listdir(saveLocation):
    wordLocation = saveLocation + '/' + word
    # Loop through every video
    for videoName in os.listdir(wordLocation): 
        usability[videoName] = None

## Load in paths

In [94]:
f = open("keyBinds.txt", "r")
line = f.readline()
while (line):
    try:
        pathMovement[line.split("_|_")[0]] = line.split("_|_")[1]
    except:
        i = 1
    line = f.readline()
f.close()

In [95]:
f = open("useability.txt", "r")
line = f.readline()
while (line):
    try:
        usability[line.split("_|_")[0]] = line.split("_|_")[1]
    except:
        print('error')
    line = f.readline()
f.close()

In [96]:
f = open("labelBool.txt", "r")
line = f.readline()
while (line):
    try:
        finishedWords[line.split("_|_")[0]] = bool(line.split("_|_")[1])
    except:
        print('error')
    line = f.readline()
f.close()

## Label Program

In [54]:
saveLocation = "D:/Thesis/Video-Auslan-sorted"
playbackSpeed = int((1/25)*1000)

 # Loop through every folder
done = False
for word in os.listdir(saveLocation):
    if done:
        break
    nextWord = False
    wordLocation = saveLocation + '/' + word
    # Loop through every video
    for videoName in os.listdir(wordLocation): 
        if nextWord or finishedWords[videoName]:
            break
        # Ensure its not corrupted
        isIn = False
        for corruptName in corrupted:
            isIn = isIn or (corruptName in videoName)
        if not isIn:
            # Load video Files
            videoLocation = wordLocation + '/' + videoName
            frames = []
            cap = cv2.VideoCapture(videoLocation)
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)
            cap.release()

            # Display video
            if len(frames) > 0:
                frameNum = 0

                # Terminal Title stuff
                extra = [videoName, str(finishedWords[videoName])]
                prevExtra = 0
                if not pathMovement[word] is None:
                    extra.append(pathMovement[word])
                if not usability[videoName] is None:
                    extra.append("Usability: " + usability[videoName])
                else:
                    extra.append("Usability: None" )
                while True:
                    # Display Frame
                    try:
                        cv2.imshow('MediaPipe Pose', frames[frameNum])
                    except:
                        print(videoName, frameNum, len(frames))
                        break

                    # Increment frame
                    frameNum += 1
                    if frameNum >= len(frames):
                        frameNum = 0

                    # Clear terminal
                    if len(extra) != prevExtra:
                        clear_output(wait=True)
                        printStr = ""
                        for wrds in extra:
                            printStr += wrds
                            printStr += " | "
                        print(printStr)
                    prevExtra = len(extra)
                    

                    # Key Press detection
                    
                    k = cv2.waitKey(playbackSpeed) & 0xFF
                    # Label Keys
                    if k == ord('8'):                           # Movement Up
                        extra.append('Up')
                        pathMovement[word] = "Up"
                    elif k == ord('2'):                         # Movement Down
                        extra.append('Down')
                        pathMovement[word] = "Down"
                    elif k == ord('5'):                         # Movement Up and Down
                        extra.append('Up and Down')
                        pathMovement[word] = "Up and Down"
                    elif k == ord('4') or k == ord('6'):        # Movement Sideways
                        extra.append('Sideways')
                        pathMovement[word] = "Sideways"
                    elif k == ord('+'):                         # Movement Side to Side
                        extra.append('Side to Side')
                        pathMovement[word] = "Side to Side"
                    elif k == ord('7') or k == ord('9'):        # Movement Away
                        extra.append('Away')
                        pathMovement[word] = "Away"
                    elif k == ord('1')or k == ord('3'):         # Movement Towards
                        extra.append('Towards')
                        pathMovement[word] = "Towards"
                    elif k == 13:                               # Movement Back and Forth
                        extra.append('Back and Forth')
                        pathMovement[word] = "Back and Forth"
                    elif k == ord('*'):                         # Movement Horizontal Circular
                        extra.append('Horizontal Circular')
                        pathMovement[word] = "Horizontal Circular"
                    elif k == ord('/'):                         # Movement Vertical Circular
                        pathMovement[word] = "Vertical Circular"
                        extra.append('Vertical Circular')
                    elif k == ord('-'):                         # Unknown
                        extra.append('Unknown')
                        pathMovement[word] = "Unknown"
                    elif k == ord('.'):                         # Local Movement
                        extra.append('Local Movement')
                        pathMovement[word] = "Local Movement"


                    # Usability of video
                    elif k == ord('z'):                         # Movement Vertical Circular
                        extra.append('Usability: 1')
                        usability[videoName] = '1'
                    elif k == ord('x'):                         # Movement Vertical Circular
                        extra.append('Usability: 2')
                        usability[videoName] = '2'
                    elif k == ord('c'):                         # Movement Vertical Circular
                        extra.append('Usability: 3')
                        usability[videoName] = '3'
                    elif k == ord(' '):                         # Movement Vertical Circular
                        extra.append('Usability: 4')
                        usability[videoName] = '4'

                    # General Keys
                    elif k == ord('q'):     # Next Variation of Sign
                        break
                    elif k == ord('t'):
                        finishedWords[videoName] = True
                    elif k == 27:           # Next Word
                        nextWord = True
                        break
                    elif k == ord(')'):     # Exit fully
                        nextWord = True
                        done = True
                        break
                cv2.destroyAllWindows()
    cv2.destroyAllWindows()
cv2.destroyAllWindows()

ALRIGHT_SLRB2c7a_63810_64380.mp4 | False | Unknown
 | Usability: None | 


In [98]:
cv2.destroyAllWindows()
f = open("keyBinds.txt", "w")
for key in pathMovement:
    if pathMovement[key] is None:
        f.write(key + "_|_" + 'None' + '\n')
    else:
        try:
            f.write(key + "_|_" + pathMovement[key] + '\n')
        except:
            print(pathMovement[key])
f.close()
pathMovement

{'ABOUT1': 'Vertical Circular\n',
 'ABOUT2': 'Vertical Circular\n',
 'ABOUT2(7)': 'Vertical Circular\n',
 'ABOUT2(B)': 'None\n',
 'ABOUT2(FALSE-START)': 'Unknown\n',
 'ACCEPT': 'Towards\n',
 'ACCOMMODATION-2H': 'Back and Forth\n',
 'ACCURATE': 'Up and Down\n',
 'ACHIEVE': 'Away\n',
 'ACTIVITY-THERE': 'Sideways\n',
 'ADULT': 'Horizontal Circular\n',
 'AFRAID': 'Back and Forth\n',
 'AFTERNOON': 'Local Movement\n',
 'AFTERNOON3': 'Towards\n',
 'AFTERNOON4': 'Towards\n',
 'AGAIN': 'Down\n',
 'AGAIN(8)': 'None\n',
 'AGAIN-2H': 'Local Movement\n',
 'AGAIN-AND-AGAIN': 'Up and Down\n',
 'AGES': 'Horizontal Circular\n',
 'AGREE': 'Sideways\n',
 'AIM': 'Unknown\n',
 'ALIVE': 'Towards\n',
 'ALL': 'Horizontal Circular\n',
 'ALL-2H': 'Away\n',
 'ALL-DAY': 'Up\n',
 'ALL-DAY-2H': 'Up\n',
 'ALL-DAY-LONG': 'Vertical Circular\n',
 'ALLOW': 'Local Movement\n',
 'ALMOST': 'Unknown\n',
 'ALMOST(FLATGO)': 'None\n',
 'ALMOST-2H': 'Unknown\n',
 'ALRIGHT': 'Unknown\n',
 'ALRIGHT-2H': 'Unknown\n',
 'ALRIGHT2': 

In [99]:
cv2.destroyAllWindows()
f = open("useability.txt", "w")
for video in usability:
    if usability[video] is None:
        f.write(video + "_|_" + 'None' + '\n')
    else:
        f.write(video + "_|_" + usability[video] + '\n')
f.close()
usability

{'ABOUT1_BFSA1c2a_1630_1990.mp4': 'None\n',
 'ABOUT1_BRCA1c2a_27365_27645.mp4': 'None\n',
 'ABOUT1_BRCA1c2a_29505_29985.mp4': 'None\n',
 'ABOUT1_SBS1A3c7a_950_1280.mp4': '3\n',
 'ABOUT2_AAPB2c7a_310_800.mp4': '2\n',
 'ABOUT2_AJPB1c2b_2490_2650.mp4': 'None\n',
 'ABOUT2_AJPB2c7a_1135_2270.mp4': '3\n',
 'ABOUT2_AMGA1c2a_1580_1820.mp4': '1\n',
 'ABOUT2_AMGA1c2a_730_1170.mp4': '3\n',
 'ABOUT2_AMW2A1c2b_890_1020.mp4': '1\n',
 'ABOUT2_BCHA2c7a_4470_5330.mp4': 'None\n',
 'ABOUT2_BCHA2c7a_5750_6050.mp4': 'None\n',
 'ABOUT2_BDLA1c2a_1800_2290.mp4': 'None\n',
 'ABOUT2_BMKB2c7a_1010_1700.mp4': '4\n',
 'ABOUT2_BRCA1c2a_9460_9980.mp4': 'None\n',
 'ABOUT2_MBHA1c2a_4750_5175.mp4': '4\n',
 'ABOUT2_MFKA1c2a_6180_6460.mp4': '2\n',
 'ABOUT2_MGCA1c2b_4880_5510.mp4': '4\n',
 'ABOUT2_MGCA1c2b_52725_53635.mp4': '4\n',
 'ABOUT2_MGCA1c2b_5840_6150.mp4': '2\n',
 'ABOUT2_MTDBA2c7a_1290_2190.mp4': '4\n',
 'ABOUT2_PGMB1c2a_1230_2110.mp4': '4\n',
 'ABOUT2_PJEB1c2a_2500_3100.mp4': '4\n',
 'ABOUT2_PNAA1c2b_1880_2660.m

In [100]:
cv2.destroyAllWindows()
f = open("labelBool.txt", "w")
for video in finishedWords:
    if finishedWords[video] is None:
        f.write(video + "_|_" + 'None' + '\n')
    else:
        f.write(video + "_|_" + str(finishedWords[video]) + '\n')
f.close()
finishedWords

{'ABOUT1_BFSA1c2a_1630_1990.mp4': True,
 'ABOUT1_BRCA1c2a_27365_27645.mp4': True,
 'ABOUT1_BRCA1c2a_29505_29985.mp4': True,
 'ABOUT1_SBS1A3c7a_950_1280.mp4': True,
 'ABOUT2_AAPB2c7a_310_800.mp4': True,
 'ABOUT2_AJPB1c2b_2490_2650.mp4': True,
 'ABOUT2_AJPB2c7a_1135_2270.mp4': True,
 'ABOUT2_AMGA1c2a_1580_1820.mp4': True,
 'ABOUT2_AMGA1c2a_730_1170.mp4': True,
 'ABOUT2_AMW2A1c2b_890_1020.mp4': True,
 'ABOUT2_BCHA2c7a_4470_5330.mp4': True,
 'ABOUT2_BCHA2c7a_5750_6050.mp4': True,
 'ABOUT2_BDLA1c2a_1800_2290.mp4': True,
 'ABOUT2_BMKB2c7a_1010_1700.mp4': True,
 'ABOUT2_BRCA1c2a_9460_9980.mp4': True,
 'ABOUT2_MBHA1c2a_4750_5175.mp4': True,
 'ABOUT2_MFKA1c2a_6180_6460.mp4': True,
 'ABOUT2_MGCA1c2b_4880_5510.mp4': True,
 'ABOUT2_MGCA1c2b_52725_53635.mp4': True,
 'ABOUT2_MGCA1c2b_5840_6150.mp4': True,
 'ABOUT2_MTDBA2c7a_1290_2190.mp4': True,
 'ABOUT2_PGMB1c2a_1230_2110.mp4': True,
 'ABOUT2_PJEB1c2a_2500_3100.mp4': True,
 'ABOUT2_PNAA1c2b_1880_2660.mp4': True,
 'ABOUT2_SBS1A1c2b_7320_7630.mp4': T

In [58]:
for video in usability:
    if usability[video] != None and pathMovement[video.split("_")[0]] != None:
        finishedWords[video] = True
finishedWords

{'ABOUT1_BFSA1c2a_1630_1990.mp4': False,
 'ABOUT1_BRCA1c2a_27365_27645.mp4': False,
 'ABOUT1_BRCA1c2a_29505_29985.mp4': False,
 'ABOUT1_SBS1A3c7a_950_1280.mp4': True,
 'ABOUT2_AAPB2c7a_310_800.mp4': True,
 'ABOUT2_AJPB1c2b_2490_2650.mp4': False,
 'ABOUT2_AJPB2c7a_1135_2270.mp4': True,
 'ABOUT2_AMGA1c2a_1580_1820.mp4': True,
 'ABOUT2_AMGA1c2a_730_1170.mp4': True,
 'ABOUT2_AMW2A1c2b_890_1020.mp4': True,
 'ABOUT2_BCHA2c7a_4470_5330.mp4': False,
 'ABOUT2_BCHA2c7a_5750_6050.mp4': False,
 'ABOUT2_BDLA1c2a_1800_2290.mp4': False,
 'ABOUT2_BMKB2c7a_1010_1700.mp4': True,
 'ABOUT2_BRCA1c2a_9460_9980.mp4': False,
 'ABOUT2_MBHA1c2a_4750_5175.mp4': True,
 'ABOUT2_MFKA1c2a_6180_6460.mp4': True,
 'ABOUT2_MGCA1c2b_4880_5510.mp4': True,
 'ABOUT2_MGCA1c2b_52725_53635.mp4': True,
 'ABOUT2_MGCA1c2b_5840_6150.mp4': True,
 'ABOUT2_MTDBA2c7a_1290_2190.mp4': True,
 'ABOUT2_PGMB1c2a_1230_2110.mp4': True,
 'ABOUT2_PJEB1c2a_2500_3100.mp4': True,
 'ABOUT2_PNAA1c2b_1880_2660.mp4': True,
 'ABOUT2_SBS1A1c2b_7320_7630

### test

In [46]:
saveLocation = "D:/Thesis/Video-Auslan-sorted" + '/' + os.listdir("D:/Thesis/Video-Auslan-sorted")[1000]
videoLocation = saveLocation + '/' + os.listdir(saveLocation)[0]

frames = []
cap = cv2.VideoCapture(videoLocation)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
cap.release()
print(width, height)
test = np.array(frames)
numPixels = test.shape[0] *test.shape[1] * test.shape[2]
sumRGB = [0,0,0]
for frame in test:
    for y in frame:
        for x in y:
            sumRGB[0] += x[0]
            sumRGB[1] += x[1]
            sumRGB[2] += x[2]
sumRGB[0] = sumRGB[0] / numPixels
sumRGB[1] = sumRGB[1] / numPixels
sumRGB[2] = sumRGB[2] / numPixels
sumRGB

480 352


[68.91214922664142, 57.23134706439394, 53.744421164772724]